<a href="https://colab.research.google.com/github/klordo/nlp_homeworks/blob/hw6/nlp_hw6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Библиотеки

In [ ]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [2]:
!pip install datasets evaluate sentence_transformers > None

In [3]:
import torch

from pathlib import Path
from sklearn.model_selection import train_test_split
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset, Dataset

In [4]:
RANDOM_STATE = 1000 - 7

# Загрузка данных

Задача - генерировать анекдоты на русском языке.

Будет использоваться датасет [russian-jokes](https://huggingface.co/datasets/IgorVolochay/russian_jokes).

In [5]:
dataset = load_dataset('IgorVolochay/russian_jokes')

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 150553
    })
})

In [7]:
dataset['train']['text'][-3:]

['Это зимой они Дед Мороз и Снегурочка. А летом - фотограф и обезьянка!',
 'До чего же суеверны зубные врачи! Постоянно предлагают тебе сплюнуть, и плевательницу ставят за твоим левым плечом.',
 'Работник мясокомбината пытался вынести со склада продукцию, и был пойман с потрохами.']

# Загрузка модели

Для обучения выбрана модель `rugpt3small_based_on_gpt2`.

In [8]:
MODEL_NAME = 'sberbank-ai/rugpt3small_based_on_gpt2'

In [9]:
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME).to('cuda:0')

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

In [11]:
tokenizer.eos_token_id = 50256
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [12]:
generator = pipeline('text-generation', model=MODEL_NAME)

In [246]:
def generate(input_text):
  input_ids = tokenizer.encode(input_text, return_tensors="pt").to('cuda:0')

  model.eval()
  with torch.no_grad():
      out = model.generate(
          input_ids,
          do_sample=True,
          num_beams=2,
          temperature=1.3,
          top_p=2.5,
          max_length=80,
          repetition_penalty=1.2,
  )
  generated_text = list(map(tokenizer.decode, out))[0]
  print(generated_text)

In [60]:
generate('Заходит в бар')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Заходит в бар, в котором он работает, и говорит:

— Ого, какая вкусная закуска…

Короче, я как-то сразу понял, что он из тех, с кем мы были знакомы.


In [89]:
generate('Встретились русский и американец')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Встретились русский и американец.

— И в чем же разница?

— В том, что он русский.

—


# Подготовка датасета

In [91]:
dataset = dataset['train'].train_test_split(test_size=0.2)

In [92]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', max_length=30, truncation=True)

In [93]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/120442 [00:00<?, ? examples/s]

Map:   0%|          | 0/30111 [00:00<?, ? examples/s]

In [94]:
train_dataset = tokenized_datasets["train"].shuffle(seed=RANDOM_STATE).select(range(5000))
eval_dataset = tokenized_datasets["test"].shuffle(seed=RANDOM_STATE).select(range(5000))

In [95]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Тренировка модели

In [97]:
OUTPUT_PATH = 'Joke_generations_runs'

In [98]:
training_args = TrainingArguments(
    output_dir=OUTPUT_PATH,         # The output directory
    overwrite_output_dir=True,        # Overwrite the content of the output dir
    num_train_epochs=40,              # number of training epochs
    per_device_train_batch_size=128,   # batch size for training
    per_device_eval_batch_size=32,    # batch size for evaluation
    warmup_steps=9,                   # number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=5
)

In [99]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(), lr=1e-5), None)
)

In [100]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=320, training_loss=3.437179946899414, metrics={'train_runtime': 1523.9456, 'train_samples_per_second': 131.238, 'train_steps_per_second': 0.21, 'total_flos': 3062016000000000.0, 'train_loss': 3.437179946899414, 'epoch': 40.0})

In [101]:
model.save_pretrained('model_joke_generator_v1.4_5000')

In [231]:
generate('Однажды')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Однажды я был молод и горяч, но в душе оставался мудрым. И однажды, проснувшись среди ночи, я подумал, что это уже произошло. А может быть, еще не все потеряно. Ведь это был последний шанс вернуть молодость. Так что, если хотите вернуться в детство, нужно просто забыть об этой страшной болезни. Только тогда жизнь войдет в свою колею.Я вышел из ванной с


In [235]:
generate('Встретились русский и американец')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Встретились русский и американец, в их руках ружья, на лицах одинаковая ухмылка, как будто им удалось свести счеты. - Ну что у вас там стряслось, мистер Холмс? - спросил русский. - Вы случайно не доктор Ватсон? - спросил американец. - Кто вы такой? - Я англичанин, - отвечал русский. - Это хорошо, потому что нам нужно найти доктора Ват


In [248]:
generate('Вовочка')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Вовочка спрашивает у мамы:- Мама, а где можно достать колбасу?- Да вот там.- И что?- А тебе ее мама скажет?- Не волнуйся, сынок, я сама ее соберу и принесу домой.- А если я ее не найду?- Я же тебе сказал, что ее нет в продаже.- Значит, ее нет в продаже?- Нет, сынок, ты же знаешь, ее нет в


Анекдоты из-за обучения стараются быть длинее и в целом также бредовые, но стиль написания стал другой.